In [1]:
import json
import math
from shapely.geometry import shape, Point, Polygon
from shapely.ops import cascaded_union
from datetime import datetime
import pandas

In [15]:
#create a dataframe to store our canopy info in
columns = ['Area','Centroid']
canopy = pandas.DataFrame(columns=columns)
canopy = canopy.fillna(0) # with 0s rather than NaNs

In [16]:
#load in the tree canopy shapes
with open('Tree Canopy 2016.geojson') as f:
    js = json.load(f)

In [17]:
#load in the BG shapes
with open('cityBG_simp20_trial.geojson') as f:
    BGjs = json.load(f)

In [18]:
#for every canopy shape, find its centroid and area (in km^2)
index = 0
for feature in js['features']:
    #print(feature)
    polygon = shape(feature['geometry'])
    area = polygon.area*6370**2  
    centroid = polygon.centroid.wkt
    #print(area, centroid)
    canopy.set_value(index, 'Area', area)
    canopy.set_value(index, 'Centroid', centroid)
    index = index + 1

In [19]:
len(js['features'])

318487

In [20]:
canopy.head()

,Area,Centroid
0,0.156865,POINT (-122.2616986291356 47.49616576331935)
1,0.152619,POINT (-122.2697749720351 47.49611623267621)
2,0.343389,POINT (-122.2618122774478 47.49621193458199)
3,0.36439,POINT (-122.2616856786135 47.49728937570325)
4,0.349257,POINT (-122.2636454862029 47.49728236174681)


In [21]:
canopy.to_csv('canopy.csv')

In [22]:
#now open excel and modify

In [23]:
#load the BG centroids
canopies = pandas.read_csv('canopy.csv')

In [24]:
#a place to save the areas to the BGs
BG_canopy_area = pandas.read_csv('BG_canopy.csv')

In [25]:
#find which BG each canopy shape is in, and add the area to the total canopy area for that block group
count = 0
for ind, canopy in canopies.iterrows():
    #print(canopy)
    point = Point(canopy['long'], canopy['lat'])
    for feature in BGjs['features']:
        #print(feature)
        polygon = shape(feature['geometry'])
        #print(polygon.contains(point))
        if polygon.contains(point):
            count = count + 1
            found_polygon = feature['properties']
            BG_id = found_polygon['gis_id']
            
            #look up current stored area for this BG
            current_area = BG_canopy_area.loc[BG_canopy_area['gis_id'] == int(BG_id), 'area'].iloc[0]
            index = BG_canopy_area.loc[BG_canopy_area['gis_id'] == int(BG_id)].index
            new_area = current_area + canopy['Area']
            BG_canopy_area.set_value(index, 'area', new_area)
            
            break

In [26]:
BG_canopy_area.head()

,gis_id,area
0,530330001001,2116.255814
1,530330001002,362.016790
2,530330001003,575.028649
3,530330001004,117.707928
4,530330001005,192.824906


In [27]:
#save to an appropriate file
BG_canopy_area.to_csv('BG_canopy_areas.csv')